In [1]:
 from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive/ADM_Final_Project

/content/gdrive/My Drive/ADM_Final_Project


In [3]:
! git clone https://github.com/Microsoft/Recommenders

fatal: destination path 'Recommenders' already exists and is not an empty directory.


In [4]:
!pip install papermill

     |████████████████████████████████| 1.1MB 9.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 747kB 49.7MB/s 
     |████████████████████████████████| 727kB 42.8MB/s 
     |████████████████████████████████| 112kB 50.4MB/s 
  Created wheel for black: filename=black-20.8b1-cp36-none-any.whl size=124185 sha256=3243d944179bce4664344e72eaccc2e7c99d69c33dc39871bb753aacf73c12e7
  Stored in directory: /root/.cache/pip/wheels/6e/10/b5/edf7359c2edd0305cce7e3f96e07daf7ce55dceac9d3ce3373
Successfully built black
  Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5


In [5]:
import os
os.chdir('/content/gdrive/My Drive/ADM_Final_Project/Recommenders')

In [6]:
%load_ext autoreload
%autoreload 2

# set the environment path to find Recommenders
import sys
sys.path.append("../../")

import logging
import numpy as np
import pandas as pd
import papermill as pm
from sklearn.preprocessing import minmax_scale
import datetime as dt

from reco_utils.common.python_utils import binarize
from reco_utils.common.timer import Timer
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    logloss,
    rsquared,
    exp_var
)
from reco_utils.recommender.sar import SAR

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
Pandas version: 1.1.5


# 1. Load Data

In [19]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
SuperStore_dataSize = '100k'


In [7]:
 data = pd.read_csv('/content/gdrive/My Drive/ADM_Final_Project/reg_lost.csv',sep=',')

# Convert the float precision to 32-bit in order to reduce memory consumption 
#data['Review'] = data['Review'].astype(np.float32)
#data['Customer_ID'] = data['Review'].astype(int)
data['Order_Date'] = pd.to_datetime(data['Order_Date'])
data['Timestamp'] =(data['Order_Date'] - dt.datetime(1970,1,1)).dt.total_seconds()

data.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,City,State,Country,Postal_Code,Market,Region,ProductID,Category,Sub-Category,ProductName,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority,Rating,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile,RFMScore,Type,Timestamp
0,42433,AG-2011-2040,2014-01-01,6/1/2014,Standard Class,8,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,NaN,Africa,Africa,1,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium,4.0,109,8,935.77550,2032.0,1,3,2,132,Regular,1.388534e+09
1,22253,IN-2011-47883,2014-01-01,8/1/2014,Standard Class,9,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,1,Office Supplies,Storage,"Tenex Lockers, Blue",120.366,3,0.1,36.036,9.72,Medium,4.5,98,3,674.94600,2254.0,1,2,2,122,Regular,1.388534e+09
2,48883,HU-2011-1220,2014-01-01,5/1/2014,Second Class,12,Annie Thurman,Consumer,Budapest,Budapest,Hungary,NaN,EMEA,EMEA,1,Office Supplies,Storage,"Tenex Lockers, Blue",66.120,4,0.0,29.640,8.17,High,4.0,15,44,9386.25960,383.0,3,4,4,344,Regular,1.388534e+09
3,11731,IT-2011-3647632,2014-01-01,5/1/2014,Second Class,20,Eugene Moren,Home Office,Stockholm,Stockholm,Sweden,NaN,EU,North,1,Office Supplies,Storage,"Tenex Lockers, Blue",44.865,3,0.5,-26.055,4.82,High,4.0,2,13,1783.23420,1553.0,4,3,3,433,Regular,1.388534e+09
4,22255,IN-2011-47883,2014-01-01,8/1/2014,Standard Class,24,Patrick O'Donnell,Consumer,Dhaka,Dhaka,Bangladesh,NaN,APAC,Central Asia,1,Office Supplies,Storage,"Tenex Lockers, Blue",113.670,5,0.1,37.770,4.70,Medium,4.0,13,65,12592.59918,234.0,3,4,4,344,Regular,1.388534e+09


In [8]:
data_reg=data[['Customer_ID','ProductID','Order_Date','Rating','Timestamp']]
data_reg.dtypes


Customer_ID             int64
ProductID               int64
Order_Date     datetime64[ns]
Rating                float64
Timestamp             float64
dtype: object

In [9]:
data_reg['Customer_ID'].unique()

array([   8,    9,   12, ..., 2945,  894, 1792])

In [10]:
data_reg[data_reg['Customer_ID']==15]

,Customer_ID,ProductID,Order_Date,Rating,Timestamp
1066,15,6,2014-07-07,4.0,1.404691e+09
6760,15,107,2014-03-01,3.5,1.393632e+09
7844,15,135,2014-06-08,0.5,1.402186e+09
12092,15,260,2014-03-18,1.5,1.395101e+09
13682,15,293,2014-06-20,5.0,1.403222e+09
13943,15,296,2014-09-20,4.5,1.411171e+09
17462,15,348,2014-05-25,1.5,1.400976e+09
17492,15,349,2014-06-25,3.5,1.403654e+09
18814,15,362,2014-03-27,4.0,1.395878e+09


## 1.2 Split the data using the python random splitter provided in utilities:

In [11]:
train, test = python_stratified_split(data_reg, ratio=0.75, col_user='Customer_ID', col_item='ProductID', seed=42)

In [12]:
print("""
Train:
Total Ratings: {train_total}
Unique Users: {train_users}
Unique Items: {train_items}

Test:
Total Ratings: {test_total}
Unique Users: {test_users}
Unique Items: {test_items}
""".format(
    train_total=len(train),
    train_users=len(train['Customer_ID'].unique()),
    train_items=len(train['ProductID'].unique()),
    test_total=len(test),
    test_users=len(test['Customer_ID'].unique()),
    test_items=len(test['ProductID'].unique()),
))


Train:
Total Ratings: 16516
Unique Users: 2832
Unique Items: 376

Test:
Total Ratings: 5183
Unique Users: 1928
Unique Items: 322



## 2 Train the SAR Model

## 2.1 Instantiate the SAR algorithm and set the index

In [13]:
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SAR(
    col_user="Customer_ID",
    col_item="ProductID",
    col_rating="Rating",
    col_timestamp="Timestamp",
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    timedecay_formula=True,
    normalize=True
)

## 2.2 Train the SAR model on our training data, and get the top-k recommendations for our testing data

In [14]:
with Timer() as train_time:
  model.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

2020-12-17 02:15:15,845 INFO     Collecting user affinity matrix
2020-12-17 02:15:15,848 INFO     Calculating time-decayed affinities
2020-12-17 02:15:15,876 INFO     Creating index columns
2020-12-17 02:15:15,912 INFO     Calculating normalization factors
2020-12-17 02:15:15,938 INFO     Building user affinity sparse matrix
2020-12-17 02:15:15,943 INFO     Calculating item co-occurrence
2020-12-17 02:15:15,955 INFO     Calculating item similarity
2020-12-17 02:15:15,957 INFO     Using jaccard based similarity
2020-12-17 02:15:15,962 INFO     Done training


Took 0.11868334199999708 seconds for training.


In [15]:
with Timer() as test_time:
    top_k = model.recommend_k_items(test, remove_seen=True)

print("Took {} seconds for prediction.".format(test_time.interval))

2020-12-17 02:15:16,005 INFO     Calculating recommendation scores
2020-12-17 02:15:16,035 INFO     Removing seen items


Took 0.07960855300001413 seconds for prediction.


In [16]:
top_k.head()

,Customer_ID,ProductID,prediction
0,2,3,0.737507
1,2,140,0.694459
2,2,102,0.643337
3,2,95,0.620406
4,2,141,0.590323


## 2.3. Evaluate how well SAR performs

In [20]:
eval_map = map_at_k(test, top_k, col_user='Customer_ID', col_item='ProductID', col_rating='Rating', k=TOP_K)
eval_ndcg = ndcg_at_k(test, top_k, col_user='Customer_ID', col_item='ProductID', col_rating='Rating', k=TOP_K)
eval_precision = precision_at_k(test, top_k, col_user='Customer_ID', col_item='ProductID', col_rating='Rating', k=TOP_K)
eval_recall = recall_at_k(test, top_k, col_user='Customer_ID', col_item='ProductID', col_rating='Rating', k=TOP_K)
eval_rmse = rmse(test, top_k, col_user='Customer_ID', col_item='ProductID', col_rating='Rating')
eval_mae = mae(test, top_k, col_user='Customer_ID', col_item='ProductID', col_rating='Rating')
eval_rsquared = rsquared(test, top_k, col_user='Customer_ID', col_item='ProductID', col_rating='Rating')
eval_exp_var = exp_var(test, top_k, col_user='Customer_ID', col_item='ProductID', col_rating='Rating')


In [21]:
positivity_threshold = 2
test_bin = test.copy()
test_bin['Rating'] = binarize(test_bin['Rating'], positivity_threshold)

top_k_prob = top_k.copy()
top_k_prob['prediction'] = minmax_scale(
    top_k_prob['prediction'].astype(float)
)

eval_logloss = logloss(test_bin, top_k_prob, col_user='Customer_ID', col_item='ProductID', col_rating='Rating')

In [22]:
print("Model:\t",
      "Top K:\t%d" % TOP_K,
      "MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall,
      "RMSE:\t%f" % eval_rmse,
      "MAE:\t%f" % eval_mae,
      "R2:\t%f" % eval_rsquared,
      "Exp var:\t%f" % eval_exp_var,
      "Logloss:\t%f" % eval_logloss,
      sep='\n')

Model:	
Top K:	10
MAP:	0.164836
NDCG:	0.254990
Precision@K:	0.100674
Recall@K:	0.392872
RMSE:	3.155019
MAE:	2.986486
R2:	-8.373662
Exp var:	-0.008044
Logloss:	0.957519


In [23]:
top_k['prediction'].sort_values()

7519     0.002905
7518     0.003007
7517     0.003018
7516     0.003042
7515     0.003142
           ...   
8520     2.053097
18050    2.058256
18240    2.067291
8010     2.109548
3350     2.166273
Name: prediction, Length: 19280, dtype: float64